In [17]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

# from attention_utils import get_activations, get_data_recurrent


In [18]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [19]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [20]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [21]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims, tokenize=True):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [22]:
def sequential_generator(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False, 
                        ):    
    
    f = open(filename)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#         print(type(labels2Idx))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize)
            if len(labels2Idx)==2:
                batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            else:
                batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
        yield ([batch_features_ft], batch_labels)

In [23]:
def train_dev_sentences(filetrain,filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    labels2Idx = {}
    train_lines = [line.strip().split("\t") for line in open(filetrain) if len(line.strip().split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev) if len(line.strip().split('\t'))==check]

    train_sentences = [x[0] for x in train_lines]
    for dataset in [train_lines, dev_lines]:
        for line in dataset:
            label = line[1]
            if label not in labels2Idx.keys():
                labels2Idx[label]= len(labels2Idx)
                
#     train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]
    train_labels = [labels2Idx[x[1]] for x in train_lines]
    dev_sentences = [x[0] for x in dev_lines]
#     dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]
    dev_labels = [labels2Idx[x[1]] for x in dev_lines]
    return (train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx)


In [43]:
file_train = '/home/jindal/notebooks/twitter_data/twitter_classes_500k_train.csv'
file_dev = '/home/jindal/notebooks/twitter_data/twitter_classes_500k_dev.csv'
train_sentences, train_labels, dev_sentences, dev_labels, labels2Id = train_dev_sentences(filetrain=file_train,filedev=file_dev, check=2)

In [44]:
print(len(dev_sentences))

100000


In [45]:
dev_labels[:10]

[421, 10, 274, 329, 134, 462, 369, 91, 273, 362]

In [46]:
print(len(dev_labels))

100000


In [47]:
n_labels = 500

In [24]:
def compile_model(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    # model_concatenated = Dropout(0.8)(model_concatenated)
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
#     new_model.summary()
    return new_model

In [71]:
print(dev_sentences[1])

RT USER_MENTION : URL / i2a32rRGUt


In [8]:
def test_model(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels_earlier:'number of original labels if loading a pretrained model',
               filename_to_save_weigths,
               batch_size, 
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               labels2Idx,
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None,
                ):
    
#     f = open(filename_to_log,"w")
    
    max_f1=0
    max_p=0
    max_r=0
    max_a=0
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(labels_earlier)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
        checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
                                     save_weights_only = True)

        for epoch in range(epochs):
            print("Epoch: %d" %(epoch+1))
            model.fit_generator(
                generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                          labels2Idx= labels2Idx,tokenize= tokenize), 
                steps_per_epoch= steps_per_epoch, epochs=1,
                validation_data = generator(filename ='/home/jindal/notebooks/twitter_data/twitter_classes_500k_dev.csv', 
                                            batch_size = batch_size, check = check_for_generator, 
                                           labels2Idx = labels2Idx, tokenize = tokenize),
                validation_steps = math.ceil(len(dev_labels) / batch_size),
                callbacks = [checkpoint]
            )

#             testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
#             for i in range(len(dev_sentences)):
#                 testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
#             results = model.predict(testset_features)


# #             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
#             predLabels = results.argmax(axis=-1)
#             devLabels = devLabels
#             f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
#             r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
#             p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
#             a = accuracy_score(devLabels, predLabels)
#             if max_f1 < f1:
#                 print("model saved. F1 is %f" %(f1))
#                 model.save(filename_to_save_weigths)
#                 max_f1 = f1
#                 max_p = p
#                 max_r = r
#                 max_a = a
#             text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
#             print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
#         to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
#         print(to_write)
#         f.write(to_write)
#         total_f1+=max_f1
#         total_prec+=max_p
#         total_acc+=max_a
#         total_recall+=max_r    
#         print("*****************************************************************************")
#     final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
#     print(final_text)
#     f.write(final_text)
#     f.close()




SyntaxError: invalid syntax (<ipython-input-8-108820a7d5bd>, line 8)

In [86]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 1
number_of_epochs = 10
twitterclasses_pretraining_log = '/home/jindal/notebooks/jindal/NER/language_model/results_pretraining_twitterclasses.txt' 
twitterclasses_pretraining_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_twitterclasses.h5'
batch_size=32*4
twitterclasses_train_file='/home/jindal/notebooks/twitter_data/twitter_classes_500k_train.csv'
tokenize = False
labels2Idx = labels2Id
f1_measure='macro'
pos_label=0
load_model_weights=False
# model_weights_file:'give filepath as str'=None, 
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=2



In [87]:
len(labels2Idx)

500

In [88]:
test_model(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels,
           number_of_tests= number_of_tests,
          number_of_epochs=number_of_epochs,
           filename_to_log=twitterclasses_pretraining_log, 
           filename_to_save_weigths=twitterclasses_pretraining_save_weigths,
           tokenize= tokenize, 
           labels2Idx= labels2Idx,
          batch_size=batch_size,
           train_file=twitterclasses_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
          nb_sequence_length=nb_sequence_length,
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

Test 1/1
Epoch: 1
Epoch 1/1
3125/3125 [==============================] - 780s 250ms/step - loss: 4.1036 - acc: 0.2424 - val_loss: 3.8253 - val_acc: 0.2802
Epoch: 2
Epoch 1/1
3125/3125 [==============================] - 810s 259ms/step - loss: 3.7542 - acc: 0.2989 - val_loss: 3.6883 - val_acc: 0.3025
Epoch: 3
Epoch 1/1
3125/3125 [==============================] - 815s 261ms/step - loss: 3.6423 - acc: 0.3167 - val_loss: 3.6047 - val_acc: 0.3182
Epoch: 4
Epoch 1/1
3125/3125 [==============================] - 811s 260ms/step - loss: 3.5797 - acc: 0.3264 - val_loss: 3.5725 - val_acc: 0.3198
Epoch: 5
Epoch 1/1
3125/3125 [==============================] - 821s 263ms/step - loss: 3.5343 - acc: 0.3336 - val_loss: 3.5454 - val_acc: 0.3239
Epoch: 6
Epoch 1/1
3125/3125 [==============================] - 793s 254ms/step - loss: 3.4973 - acc: 0.3394 - val_loss: 3.5259 - val_acc: 0.3284
Epoch: 7
Epoch 1/1
3125/3125 [==============================] - 785s 251ms/step - loss: 3.4718 - acc: 0.3430 - val_

# TRANSFER LEARNING HERE


In [37]:
def test_model_tl(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               filename_to_save_weigths,
               batch_size, 
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None,
                ):
    
    f = open(filename_to_log,"w")
    
   
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(1000)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)
                print("removing top layer")
                model.layers.pop()
                output = Dense(2, activation = 'softmax')(model.layers[-1].output)
                final_model = Model(inputs=model.input, outputs=[output])
                final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
#                 for layer in final_model.layers:
#                     print(layer.name)

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
#         checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
#                                      save_weights_only = True)


        max_f1=0
        max_p=0
        max_r=0
        max_a=0
        for epoch in range(epochs):
            print("Epoch: %d" %(epoch+1))
            final_model.fit_generator(
                generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                          labels2Idx= labels2Idx,tokenize= tokenize), 
                steps_per_epoch= steps_per_epoch, epochs=1,
#                 validation_data = generator(filename ='/home/jindal/notebooks/twitter_data/twitter_classes_500k_dev.csv', 
#                                             batch_size = batch_size, check = check_for_generator, 
#                                            labels2Idx = labels2Idx, tokenize = tokenize),
#                 validation_steps = math.ceil(len(dev_labels) / batch_size),
#                 callbacks = [checkpoint]
            )

            testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
            for i in range(len(dev_sentences)):
                testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
            results = final_model.predict(testset_features)


#             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
            predLabels = results.argmax(axis=-1)
            devLabels = devLabels
            f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
            r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            a = accuracy_score(devLabels, predLabels)
            if max_f1 < f1:
                print("model saved. F1 is %f" %(f1))
                final_model.save(filename_to_save_weigths)
                max_f1 = f1
                max_p = p
                max_r = r
                max_a = a
            text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
            print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        f.write(to_write)
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    f.write(final_text)
    f.close()




In [26]:
n_labels =2

In [27]:
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(filetrain='/home/gwiedemann/notebooks/OffLang/sample_train.txt',
                   filedev='/home/gwiedemann/notebooks/OffLang/sample_dev.txt', check=3)

In [28]:
print(dev_sentences[0])
print(dev_labels[:20])

@FilmElf_yt Diese ganzen leute die du angesprochen hast sind untermenschen ja aber das alte Deutschland und deutsche volk waren ehrenwerte leute und ein stolzes Land
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]


In [38]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 5
number_of_epochs = 50
twitterclasses_tl_log = '/home/jindal/notebooks/jindal/NER/language_model/results_tl_twitterclasses7m_1000.txt' 
twitterclasses_tl_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_twitterclasses7m_1000.h5'
batch_size=32
twitterclasses_tl_train_file='/home/gwiedemann/notebooks/OffLang/sample_train.txt'
f1_measure='binary'
pos_label=1
load_model_weights=True
model_weights_file = '/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_twitterclasses7m_1000.h5'
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=3

In [ ]:
test_model_tl(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, 
           filename_to_log=twitterclasses_tl_log, 
           filename_to_save_weigths=twitterclasses_tl_save_weigths,
           batch_size=batch_size,
           train_file=twitterclasses_tl_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
           model_weights_file = model_weights_file, 
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

Test 1/5
removing top layer
Epoch: 1
Epoch 1/1
132/132 [==============================] - 35s 263ms/step - loss: 0.5934 - acc: 0.6927
model saved. F1 is 0.644000
Test-Data: Prec: 0.691, Rec: 0.603, F1: 0.644, Acc: 0.780
Epoch: 2
Epoch 1/1
132/132 [==============================] - 30s 231ms/step - loss: 0.4883 - acc: 0.7640
model saved. F1 is 0.662698
Test-Data: Prec: 0.705, Rec: 0.625, F1: 0.663, Acc: 0.790
Epoch: 3
Epoch 1/1
132/132 [==============================] - 31s 233ms/step - loss: 0.4513 - acc: 0.7796
Test-Data: Prec: 0.707, Rec: 0.614, F1: 0.657, Acc: 0.788
Epoch: 4
Epoch 1/1
132/132 [==============================] - 30s 229ms/step - loss: 0.4044 - acc: 0.8078
model saved. F1 is 0.672098
Test-Data: Prec: 0.737, Rec: 0.618, F1: 0.672, Acc: 0.801
Epoch: 5
Epoch 1/1
132/132 [==============================] - 30s 229ms/step - loss: 0.3860 - acc: 0.8203
model saved. F1 is 0.677104
Test-Data: Prec: 0.709, Rec: 0.648, F1: 0.677, Acc: 0.796
Epoch: 6
Epoch 1/1
132/132 [============

# gradual unfreezing


In [52]:
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(filetrain='/home/gwiedemann/notebooks/OffLang/sample_train.txt',
                   filedev='/home/gwiedemann/notebooks/OffLang/sample_dev.txt', check=3)

In [55]:
print(dev_sentences[0])
print(len(dev_sentences))
print(len(dev_labels))

@FilmElf_yt Diese ganzen leute die du angesprochen hast sind untermenschen ja aber das alte Deutschland und deutsche volk waren ehrenwerte leute und ein stolzes Land
808
808


In [56]:
# list of tuples. Every tuple contains range of layers which need to be unfrozen. Rest all are frozen
single_unfreeze_bottom_up = [(18, 18), (17, 16), (15, 3), (2, 1), (18,1)] 
single_unfreeze_top_down = [(18, 18),   (2, 1),(15, 3), (17, 16), (18,1)]
all_unfreeze = [(18,1)]
gradual_unfreezing = [(18,18), (16,18), (3,18), (1,18)]


In [65]:
def test_model_tl_unfreezing(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels2Idx,
               filename_to_save_weigths,
               batch_size, 
               unfreezing_strategy: 'list containing a tuple of indices to unfreeze at each step',
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None):
    
    f = open(filename_to_log, 'w', encoding='utf-8')
    
   
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(1000)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)
                print("removing top layer")
                model.layers.pop()
                output = Dense(2, activation = 'softmax')(model.layers[-1].output)
                final_model = Model(inputs=model.input, outputs=[output])

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
#         checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
#                                      save_weights_only = True)


        max_f1=0
        max_p=0
        max_r=0
        max_a=0
        
        # load pretrained weights
        # model.compile
        # save tmp weights
        # iterate over layers
        #    load tmp weights
        #    iterate over epochs
        #        unfreeze top frozen layer
        #        save best model as tmp weights
        
        
        final_model.save(filename_to_save_weigths)
        
        # layers_to_unfreeze = [18, 16, 3, 1]
        
        for ulayer in unfreezing_strategy:
            print("unfreezing " + final_model.layers[ulayer[0]].name)
            print("---------------------------------------")
            final_model.load_weights(filename_to_save_weigths)            
            for i, layer in enumerate(final_model.layers):
                
                # TF strategy: gradual unfreezing
                #if i >= ulayer:
                #    layer.trainable = True
                #else:
                #    layer.trainable = False
                # 
                ## TF strategy: single
                
                if i >= ulayer[1] and i <= ulayer[0]:
                    layer.trainable = True
                else:
                    layer.trainable = False
                    
                print(str(i) + ' ' + layer.name + ' ' + str(layer.trainable))
            final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
        
            for epoch in range(epochs):
                print("Epoch: %d/%d" %(epoch+1, epochs))
                final_model.fit_generator(
                    generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                              labels2Idx= labels2Idx,tokenize= tokenize), 
                    steps_per_epoch= steps_per_epoch, epochs=1
                )

                testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
                for i in range(len(dev_sentences)):
                    testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
                results = final_model.predict(testset_features)

                predLabels = results.argmax(axis=-1)
                devLabels = devLabels
                f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
                r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                a = accuracy_score(devLabels, predLabels)
                if max_f1 < f1:
                    print("model saved. F1 is %f" %(f1))
                    final_model.save(filename_to_save_weigths)
                    max_f1 = f1
                    max_p = p
                    max_r = r
                    max_a = a
                text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
                print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        f.write(to_write)
        f.close()
        f = open(filename_to_log, 'a', encoding='utf-8')
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    f.write(final_text)
    f.close()

In [68]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 10
number_of_epochs = 50
labels2Id = labels2Idx
twitterclasses_tl_log = '/home/jindal/notebooks/jindal/NER/language_model/results_tl_twitterclasses7m_1000_suf_td.txt' 
twitterclasses_tl_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_twitterclasses7m_1000_suf_td.h5'
batch_size=32
twitterclasses_tl_train_file='/home/gwiedemann/notebooks/OffLang/sample_train.txt'
f1_measure='binary'
pos_label=1
unfreeze_strategy = single_unfreeze_top_down
load_model_weights=True
model_weights_file = '/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_twitterclasses7m_1000.h5'
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=3

In [ ]:
test_model_tl_unfreezing(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, 
           filename_to_log=twitterclasses_tl_log, 
           labels2Idx = labels2Id,
           filename_to_save_weigths=twitterclasses_tl_save_weigths,
           batch_size=batch_size,
           unfreezing_strategy = unfreeze_strategy,       
           train_file=twitterclasses_tl_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
           model_weights_file = model_weights_file, 
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

Test 1/10
removing top layer
unfreezing dense_69
---------------------------------------
0 input_24 False
1 bidirectional_24 False
2 leaky_re_lu_116 False
3 conv1d_70 False
4 conv1d_71 False
5 conv1d_72 False
6 leaky_re_lu_117 False
7 leaky_re_lu_118 False
8 leaky_re_lu_119 False
9 global_max_pooling1d_70 False
10 global_max_pooling1d_71 False
11 global_max_pooling1d_72 False
12 dropout_70 False
13 dropout_71 False
14 dropout_72 False
15 concatenate_24 False
16 dense_67 False
17 leaky_re_lu_120 False
18 dense_69 True
Epoch: 1/50
Epoch 1/1
132/132 [==============================] - 14s 105ms/step - loss: 0.6583 - acc: 0.6432
model saved. F1 is 0.493274
Test-Data: Prec: 0.615, Rec: 0.412, F1: 0.493, Acc: 0.720
Epoch: 2/50
Epoch 1/1
132/132 [==============================] - 10s 76ms/step - loss: 0.5888 - acc: 0.6911
model saved. F1 is 0.502304
Test-Data: Prec: 0.653, Rec: 0.408, F1: 0.502, Acc: 0.733
Epoch: 3/50
Epoch 1/1
132/132 [==============================] - 10s 73ms/step - loss: 0

132/132 [==============================] - 30s 224ms/step - loss: 0.1295 - acc: 0.9515
Test-Data: Prec: 0.729, Rec: 0.745, F1: 0.737, Acc: 0.824
Epoch: 40/50
Epoch 1/1
132/132 [==============================] - 32s 242ms/step - loss: 0.1210 - acc: 0.9550
model saved. F1 is 0.741410
Test-Data: Prec: 0.717, Rec: 0.768, F1: 0.741, Acc: 0.823
Epoch: 41/50
Epoch 1/1
132/132 [==============================] - 33s 247ms/step - loss: 0.1179 - acc: 0.9545
Test-Data: Prec: 0.717, Rec: 0.760, F1: 0.738, Acc: 0.822
Epoch: 42/50
Epoch 1/1
132/132 [==============================] - 33s 249ms/step - loss: 0.1231 - acc: 0.9510
Test-Data: Prec: 0.681, Rec: 0.783, F1: 0.728, Acc: 0.807
Epoch: 43/50
Epoch 1/1
132/132 [==============================] - 33s 252ms/step - loss: 0.1187 - acc: 0.9550
model saved. F1 is 0.743772
Test-Data: Prec: 0.708, Rec: 0.783, F1: 0.744, Acc: 0.822
Epoch: 44/50
Epoch 1/1
132/132 [==============================] - 33s 250ms/step - loss: 0.1108 - acc: 0.9595
model saved. F1 i

132/132 [==============================] - 10s 77ms/step - loss: 0.0955 - acc: 0.9590
Test-Data: Prec: 0.711, Rec: 0.783, F1: 0.745, Acc: 0.823
Epoch: 31/50
Epoch 1/1
132/132 [==============================] - 10s 76ms/step - loss: 0.0964 - acc: 0.9624
Test-Data: Prec: 0.716, Rec: 0.783, F1: 0.748, Acc: 0.825
Epoch: 32/50
Epoch 1/1
132/132 [==============================] - 10s 78ms/step - loss: 0.0875 - acc: 0.9671
Test-Data: Prec: 0.701, Rec: 0.790, F1: 0.743, Acc: 0.819
Epoch: 33/50
Epoch 1/1
132/132 [==============================] - 10s 73ms/step - loss: 0.1008 - acc: 0.9624
Test-Data: Prec: 0.693, Rec: 0.801, F1: 0.743, Acc: 0.817
Epoch: 34/50
Epoch 1/1
132/132 [==============================] - 9s 71ms/step - loss: 0.0997 - acc: 0.9598
Test-Data: Prec: 0.687, Rec: 0.805, F1: 0.741, Acc: 0.814
Epoch: 35/50
Epoch 1/1
132/132 [==============================] - 10s 73ms/step - loss: 0.0916 - acc: 0.9650
Test-Data: Prec: 0.701, Rec: 0.790, F1: 0.743, Acc: 0.819
Epoch: 36/50
Epoch 1/1

132/132 [==============================] - 10s 76ms/step - loss: 0.5356 - acc: 0.7256
Test-Data: Prec: 0.695, Rec: 0.393, F1: 0.502, Acc: 0.743
Epoch: 21/50
Epoch 1/1
132/132 [==============================] - 10s 77ms/step - loss: 0.5419 - acc: 0.7188
Test-Data: Prec: 0.705, Rec: 0.464, F1: 0.560, Acc: 0.759
Epoch: 22/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step - loss: 0.5391 - acc: 0.7249
Test-Data: Prec: 0.702, Rec: 0.423, F1: 0.528, Acc: 0.750
Epoch: 23/50
Epoch 1/1
132/132 [==============================] - 10s 74ms/step - loss: 0.5365 - acc: 0.7185
Test-Data: Prec: 0.693, Rec: 0.464, F1: 0.556, Acc: 0.755
Epoch: 24/50
Epoch 1/1
132/132 [==============================] - 10s 78ms/step - loss: 0.5441 - acc: 0.7105
Test-Data: Prec: 0.670, Rec: 0.494, F1: 0.569, Acc: 0.752
Epoch: 25/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step - loss: 0.5402 - acc: 0.7188
Test-Data: Prec: 0.682, Rec: 0.457, F1: 0.547, Acc: 0.750
Epoch: 26/50
Epoch 1/

132/132 [==============================] - 30s 229ms/step - loss: 0.3028 - acc: 0.8686
model saved. F1 is 0.735135
Test-Data: Prec: 0.708, Rec: 0.764, F1: 0.735, Acc: 0.818
Epoch: 16/50
Epoch 1/1
132/132 [==============================] - 31s 231ms/step - loss: 0.2888 - acc: 0.8731
Test-Data: Prec: 0.720, Rec: 0.742, F1: 0.731, Acc: 0.819
Epoch: 17/50
Epoch 1/1
132/132 [==============================] - 30s 231ms/step - loss: 0.2845 - acc: 0.8767
Test-Data: Prec: 0.705, Rec: 0.715, F1: 0.710, Acc: 0.807
Epoch: 18/50
Epoch 1/1
132/132 [==============================] - 30s 231ms/step - loss: 0.2759 - acc: 0.8826
Test-Data: Prec: 0.717, Rec: 0.749, F1: 0.733, Acc: 0.819
Epoch: 19/50
Epoch 1/1
132/132 [==============================] - 31s 232ms/step - loss: 0.2615 - acc: 0.8849
Test-Data: Prec: 0.705, Rec: 0.760, F1: 0.732, Acc: 0.816
Epoch: 20/50
Epoch 1/1
132/132 [==============================] - 31s 233ms/step - loss: 0.2568 - acc: 0.8873
Test-Data: Prec: 0.722, Rec: 0.738, F1: 0.730

...

Epoch 1/1
132/132 [==============================] - 15s 116ms/step - loss: 0.6564 - acc: 0.6548
model saved. F1 is 0.449074
Test-Data: Prec: 0.588, Rec: 0.363, F1: 0.449, Acc: 0.705
Epoch: 2/50
Epoch 1/1
132/132 [==============================] - 9s 70ms/step - loss: 0.5650 - acc: 0.7109
model saved. F1 is 0.506849
Test-Data: Prec: 0.649, Rec: 0.416, F1: 0.507, Acc: 0.733
Epoch: 3/50
Epoch 1/1
132/132 [==============================] - 9s 69ms/step - loss: 0.5599 - acc: 0.7045
model saved. F1 is 0.533030
Test-Data: Prec: 0.680, Rec: 0.438, F1: 0.533, Acc: 0.746
Epoch: 4/50
Epoch 1/1
132/132 [==============================] - 9s 69ms/step - loss: 0.5502 - acc: 0.7145
Test-Data: Prec: 0.683, Rec: 0.412, F1: 0.514, Acc: 0.743
Epoch: 5/50
Epoch 1/1
132/132 [==============================] - 9s 69ms/step - loss: 0.5498 - acc: 0.7183
model saved. F1 is 0.552809
Test-Data: Prec: 0.691, Rec: 0.461, F1: 0.553, Acc: 0.754
Epoch: 6/50
Epoch 1/1
132/132 [==============================] - 9s 70ms/

132/132 [==============================] - 28s 212ms/step - loss: 0.1184 - acc: 0.9560
Test-Data: Prec: 0.708, Rec: 0.727, F1: 0.717, Acc: 0.811
Epoch: 44/50
Epoch 1/1
132/132 [==============================] - 28s 212ms/step - loss: 0.1175 - acc: 0.9524
Test-Data: Prec: 0.710, Rec: 0.734, F1: 0.722, Acc: 0.813
Epoch: 45/50
Epoch 1/1
132/132 [==============================] - 28s 213ms/step - loss: 0.1149 - acc: 0.9545
Test-Data: Prec: 0.697, Rec: 0.742, F1: 0.719, Acc: 0.808
Epoch: 46/50
Epoch 1/1
132/132 [==============================] - 28s 212ms/step - loss: 0.1119 - acc: 0.9574
Test-Data: Prec: 0.705, Rec: 0.682, F1: 0.693, Acc: 0.801
Epoch: 47/50
Epoch 1/1
132/132 [==============================] - 28s 213ms/step - loss: 0.1065 - acc: 0.9583
Test-Data: Prec: 0.697, Rec: 0.715, F1: 0.706, Acc: 0.803
Epoch: 48/50
Epoch 1/1
132/132 [==============================] - 28s 213ms/step - loss: 0.0983 - acc: 0.9657
Test-Data: Prec: 0.702, Rec: 0.697, F1: 0.699, Acc: 0.802
Epoch: 49/50
Ep

132/132 [==============================] - 10s 72ms/step - loss: 0.0448 - acc: 0.9834
Test-Data: Prec: 0.676, Rec: 0.734, F1: 0.704, Acc: 0.796
Epoch: 40/50
Epoch 1/1
132/132 [==============================] - 10s 73ms/step - loss: 0.0463 - acc: 0.9825
Test-Data: Prec: 0.668, Rec: 0.760, F1: 0.711, Acc: 0.796
Epoch: 41/50
Epoch 1/1
132/132 [==============================] - 10s 72ms/step - loss: 0.0427 - acc: 0.9856
Test-Data: Prec: 0.673, Rec: 0.742, F1: 0.706, Acc: 0.796
Epoch: 42/50
Epoch 1/1
132/132 [==============================] - 10s 73ms/step - loss: 0.0514 - acc: 0.9801
Test-Data: Prec: 0.661, Rec: 0.775, F1: 0.714, Acc: 0.795
Epoch: 43/50
Epoch 1/1
132/132 [==============================] - 10s 72ms/step - loss: 0.0440 - acc: 0.9832
Test-Data: Prec: 0.669, Rec: 0.764, F1: 0.713, Acc: 0.797
Epoch: 44/50
Epoch 1/1
132/132 [==============================] - 10s 73ms/step - loss: 0.0426 - acc: 0.9865
Test-Data: Prec: 0.663, Rec: 0.760, F1: 0.709, Acc: 0.793
Epoch: 45/50
Epoch 1/

132/132 [==============================] - 9s 69ms/step - loss: 0.0937 - acc: 0.9661
Test-Data: Prec: 0.669, Rec: 0.787, F1: 0.723, Acc: 0.801
Epoch: 36/50
Epoch 1/1
132/132 [==============================] - 9s 68ms/step - loss: 0.0841 - acc: 0.9666
Test-Data: Prec: 0.678, Rec: 0.779, F1: 0.725, Acc: 0.804
Epoch: 37/50
Epoch 1/1
132/132 [==============================] - 9s 68ms/step - loss: 0.0910 - acc: 0.9673
Test-Data: Prec: 0.678, Rec: 0.749, F1: 0.712, Acc: 0.800
Epoch: 38/50
Epoch 1/1
132/132 [==============================] - 9s 68ms/step - loss: 0.0836 - acc: 0.9673
Test-Data: Prec: 0.674, Rec: 0.760, F1: 0.715, Acc: 0.800
Epoch: 39/50
Epoch 1/1
132/132 [==============================] - 9s 70ms/step - loss: 0.0895 - acc: 0.9669
Test-Data: Prec: 0.678, Rec: 0.749, F1: 0.712, Acc: 0.800
Epoch: 40/50
Epoch 1/1
132/132 [==============================] - 9s 69ms/step - loss: 0.0821 - acc: 0.9692
Test-Data: Prec: 0.683, Rec: 0.768, F1: 0.723, Acc: 0.806
Epoch: 41/50
Epoch 1/1
132/

132/132 [==============================] - 9s 70ms/step - loss: 0.5435 - acc: 0.7197
model saved. F1 is 0.572687
Test-Data: Prec: 0.695, Rec: 0.487, F1: 0.573, Acc: 0.760
Epoch: 26/50
Epoch 1/1
132/132 [==============================] - 9s 70ms/step - loss: 0.5411 - acc: 0.7178
Test-Data: Prec: 0.688, Rec: 0.453, F1: 0.546, Acc: 0.751
Epoch: 27/50
Epoch 1/1
132/132 [==============================] - 9s 71ms/step - loss: 0.5336 - acc: 0.7204
Test-Data: Prec: 0.706, Rec: 0.404, F1: 0.514, Acc: 0.748
Epoch: 28/50
Epoch 1/1
132/132 [==============================] - 9s 70ms/step - loss: 0.5364 - acc: 0.7161
Test-Data: Prec: 0.663, Rec: 0.479, F1: 0.557, Acc: 0.748
Epoch: 29/50
Epoch 1/1
132/132 [==============================] - 9s 69ms/step - loss: 0.5385 - acc: 0.7206
Test-Data: Prec: 0.711, Rec: 0.461, F1: 0.559, Acc: 0.760
Epoch: 30/50
Epoch 1/1
132/132 [==============================] - 9s 69ms/step - loss: 0.5439 - acc: 0.7169
Test-Data: Prec: 0.696, Rec: 0.438, F1: 0.538, Acc: 0.751

In [70]:
# list of tuples. Every tuple contains range of layers which need to be unfrozen. Rest all are frozen
single_unfreeze_bottom_up = [(18, 18), (17, 16), (15, 3), (2, 1), (18,1)] 
single_unfreeze_top_down = [(18, 18),   (2, 1),(15, 3), (17, 16), (18,1)]
all_unfreeze = [(18,1)]
gradual_unfreezing = [(18,18), (16,18), (3,18), (1,18)]

strings =['suf_bu','all_unfreeze','gradual_unfreeze']
unfeeze_strategy = [single_unfreeze_bottom_up, all_unfreeze, gradual_unfreezing]

In [ ]:
for i in range(len(strings)):
    string = strings[i]
    print("approach: %s" %(string))
    
    generator = sequential_generator
    train_sentences = train_sentences
    devLabels = dev_labels
    number_of_tests = 10
    number_of_epochs = 50
    labels2Id = labels2Idx
    twitterclasses_tl_log = '/home/jindal/notebooks/jindal/NER/language_model/results_tl_twitterclasses7m_1000_' +string+'.txt' 
    print("log file: %s" %(twitterclasses_tl_log))
    twitterclasses_tl_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_twitterclasses7m_1000_'+string+'.h5'
    print("save weights file: %s" %(twitterclasses_tl_save_weigths))
    batch_size=32
    twitterclasses_tl_train_file='/home/gwiedemann/notebooks/OffLang/sample_train.txt'
    f1_measure='binary'
    pos_label=1
    unfreeze_strategy = single_unfreeze_top_down
    load_model_weights=True
    model_weights_file = '/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_twitterclasses7m_1000.h5'
    nb_sequence_length = nb_sequence_length
    nb_embedding_dims= nb_embedding_dims
    check_for_generator=3
    
    test_model_tl_unfreezing(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, 
           filename_to_log=twitterclasses_tl_log, 
           labels2Idx = labels2Id,
           filename_to_save_weigths=twitterclasses_tl_save_weigths,
           batch_size=batch_size,
           unfreezing_strategy = unfreeze_strategy,       
           train_file=twitterclasses_tl_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
           model_weights_file = model_weights_file, 
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

approach: suf_bu
log file: /home/jindal/notebooks/jindal/NER/language_model/results_tl_twitterclasses7m_1000_suf_bu.txt
save weights file: /home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_twitterclasses7m_1000_suf_bu.h5
Test 1/10
